In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
key_path = "./creds/service_acc_auth.json"
credentials = service_account.Credentials.from_service_account_file(key_path, scopes = ["https://www.googleapis.com/auth/cloud-platform"])
bigquery_client = bigquery.Client(credentials=credentials, project=credentials.project_id)

In [2]:
# query new data
past_3_weeks = """
SELECT * , 
FROM `scanning-database.scanning_data_backup.W35`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W36`
UNION ALL
SELECT * , 
FROM `scanning-database.scanning_data_backup.W37`
"""

In [3]:
# Overwriting old PowerBI source data
job_config = bigquery.QueryJobConfig(maximum_bytes_billed=10**10) # set query limit to 10G
query_job = bigquery_client.query(past_3_weeks, job_config)  # API request
rows = query_job.result()  # Waits for query to finish
df_p_3 = rows.to_dataframe()

In [4]:
df_p_3 ## past 3 weeks

,Start_Time,Finish_Time,Activity,Seq_Code,Recipe_Name,Break_Reasons,Missing_Ingredients,Kitting_Line,Assembly_Batch,Event_Shift,Team_Leader,Pickers_Count,Time_Consumption,Week
0,2020-08-27 08:57:45+00:00,2020-08-27 08:58:12+00:00,TBD,None,L2,None,None,KL1,None,Thursday Morning,iXUsr_Strzeletz(6101),3,0.45,2020-36
1,2020-08-27 13:18:22+00:00,2020-08-27 13:18:40+00:00,TBD,None,L2,None,None,KL1,None,Thursday Morning,iXUsr_Pontoh(6763),1,0.30,2020-36
2,2020-08-28 15:10:17+00:00,2020-08-28 15:11:20+00:00,TBD,None,L2,None,None,KL1,None,Friday Afternoon,iXUsr_Ushida(8909),3,1.05,2020-36
3,2020-08-29 15:38:22+00:00,2020-08-29 15:42:01+00:00,TBD,None,L4,None,None,KL1,None,Saturday Afternoon,iXUsr_Ayob(9830),2,3.65,2020-36
4,2020-08-31 11:26:16+00:00,2020-08-31 11:26:40+00:00,TBD,None,L4,None,None,KL1,None,Monday Morning,iXUsr_Normie(7460),2,0.40,2020-36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25281,2020-08-20 13:10:50+00:00,2020-08-20 13:12:52+00:00,Break,None,O2,move to other line,None,KL20,None,Thursday Morning,Dummy;,4,2.03,2020-35
25282,2020-08-20 14:56:46+00:00,2020-08-20 18:42:44+00:00,Break,None,SG,move to other line,None,KL20,None,Thursday Afternoon,iXUsr_Good(8522),3,225.97,2020-35
25283,2020-08-21 18:50:19+00:00,2020-08-22 06:51:09+00:00,Break,None,SG,move to other line,None,KL20,None,Saturday Morning,iXUsr_Thumlert(8509),2,720.83,2020-35
25284,2020-08-22 18:06:19+00:00,2020-08-23 16:19:35+00:00,Break,None,O4,move to other line,None,KL20,None,Sunday Afternoon,iXUsr_Zhou(9355),3,1333.27,2020-35


In [5]:
import datetime as dt
import pandas as pd

In [6]:
# actual_start_time = test_group.sort_values(["Finish_Time"]).iloc[0]['Finish_Time'].tz_localize(None)
# current_date = dt.datetime.date(actual_start_time)
# morning_time = dt.time(6, 5) # morning shift suppose to start at 6:05 
# shift_start_time = dt.datetime.combine(current_date, morning_time)

In [7]:
# (actual_start_time - shift_start_time).seconds / 60

In [8]:
# index_and_sort = test_group.set_index('Finish_Time').sort_index()
# current_date = dt.datetime.date(index_and_sort.index[0])
# morning_time = dt.time(14, 0)
# shift_start_time = dt.datetime.combine(current_date, morning_time)
# print(shift_start_time)
# index_and_sort.index.get_loc(shift_start_time, method='nearest')

In [9]:
def find_start_time(grouped_df):
    week_day = grouped_df['Event_Shift'].str.split().iloc[0][0]
    shift = grouped_df['Event_Shift'].str.split().iloc[0][1]
    first_event = grouped_df.sort_values(["Finish_Time"]).iloc[0]
    actual_start_time = first_event['Finish_Time'].tz_localize(None)
    current_date = dt.datetime.date(actual_start_time)
    morning_time = dt.time(6, 0) # morning shift suppose to start at 6:05 
    afternoon_time = dt.time(14, 15) # afternoon shift start at 14:10
    wed_time = dt.time(12, 0) # Wednesday shift suppose to start at 12:00 
    if week_day in ['Thursday', 'Friday', 'Saturday', 'Sunday']:
        if (shift == "Afternoon"):
            shift_start_time = dt.datetime.combine(current_date, afternoon_time)
            lateness = (actual_start_time - shift_start_time).seconds / 60

        elif (shift == "Morning"):
            shift_start_time = dt.datetime.combine(current_date, morning_time)
            lateness = (actual_start_time - shift_start_time).seconds / 60
        else:
            raise Exception("Oops, Event_Shift Error!")

    elif week_day in ['Monday']:
        shift_start_time = dt.datetime.combine(current_date, morning_time)
        lateness = (actual_start_time - shift_start_time).seconds / 60

    elif week_day in ['Wednesday']:
        shift_start_time = dt.datetime.combine(current_date, wed_time)
        lateness = (actual_start_time - shift_start_time).seconds / 60        


    return (lateness if (lateness < 30) else 'not on kitting at the production begin')


In [10]:
df_lateness = pd.DataFrame( {'lateness': df_p_3.groupby(["Week", "Team_Leader", "Event_Shift"]).apply(find_start_time)})

In [11]:
df_lateness

lateness
Week    Team_Leader       Event_Shift                                                
2020-35 Bayu Agus Alfian  Sunday Morning                                      20.4667
                          Thursday Morning                                    11.7167
        Dummy             Saturday Afternoon   not on kitting at the production begin
                          Sunday Afternoon     not on kitting at the production begin
        Dummy;            Friday Morning       not on kitting at the production begin
...                                                                               ...
2020-37 iXUsr_Yuhan(8855) Friday Afternoon                                    15.4167
                          Saturday Afternoon                                      9.9
                          Thursday Afternoon                                  21.6667
                          Wednesday Afternoon  not on kitting at the production begin
                          Wednesday Morning                                   24.3667

[379 rows x 1 columns]

In [12]:
df_new = df_lateness[df_lateness['lateness'] != "not on kitting at the production begin"].reset_index()
df_new

,Week,Team_Leader,Event_Shift,lateness
0,2020-35,Bayu Agus Alfian,Sunday Morning,20.4667
1,2020-35,Bayu Agus Alfian,Thursday Morning,11.7167
2,2020-35,JAMES RAY ALAY-AY;,Friday Afternoon,12.6667
3,2020-35,JAMES RAY ALAY-AY;,Sunday Afternoon,12.2167
4,2020-35,James Ray Alay-ay;,Thursday Afternoon,12.6
...,...,...,...,...
273,2020-37,iXUsr_Ye'Rafael(8520),Wednesday Morning,13.0333
274,2020-37,iXUsr_Yuhan(8855),Friday Afternoon,15.4167
275,2020-37,iXUsr_Yuhan(8855),Saturday Afternoon,9.9
276,2020-37,iXUsr_Yuhan(8855),Thursday Afternoon,21.6667


In [13]:
df_new.replace({'Wednesday Morning': 'Wednesday', 'Monday Morning': 'Monday'}, inplace = True)

In [14]:
df_new.to_excel("Week_37.xlsx", sheet_name = 'Week 37', index = False) 
df_lateness.to_excel("Past_3_weeks.xlsx", sheet_name = 'past 3 weeks') 

In [15]:
df_W37 = df_new[df_new["Week"] == '2020-37'][["Team_Leader", "Event_Shift", 'lateness']]
df_W37

,Team_Leader,Event_Shift,lateness
187,BAYU,Sunday Afternoon,4.366667
188,BAYU,Sunday Morning,21.616667
189,Bayu Agus Alfian,Thursday Morning,11.250000
190,JAMES RAY ALAY-AY;,Friday Afternoon,22.133333
191,JAMES RAY ALAY-AY;,Saturday Afternoon,17.066667
...,...,...,...
273,iXUsr_Ye'Rafael(8520),Wednesday,13.033333
274,iXUsr_Yuhan(8855),Friday Afternoon,15.416667
275,iXUsr_Yuhan(8855),Saturday Afternoon,9.900000
276,iXUsr_Yuhan(8855),Thursday Afternoon,21.666667


In [25]:
df_temp1 = df_W37.groupby("Team_Leader").mean()["lateness"]

In [26]:
df_temp2 = df_W37.groupby("Team_Leader").count()['Event_Shift']

In [29]:
df_out = pd.concat([df_temp1, df_temp2], axis = 1)
df_out.to_excel('output.xlsx')